# Pressure Vessel Design Problem

In [7]:
import numpy as np
import random
import math
import time
# from numpy import  uniform

# Solution class to store optimization results
class solution:
    def __init__(self):
        self.best = float('inf')
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = ""
        self.endTime = ""
        self.executionTime = 0

# Objective function class for Pressure Vessel Design
class ObjectiveFunction:
    def __init__(self):
        pass
    
    def evaluate(self, x):
        # Objective function
        cost = 0.6224 * x[0] * x[2] * x[3] + 1.7781 * x[1] * x[2]**2 + 3.1661 * x[0]**2 * x[3] + 19.84 * x[0]**2 * x[2]
        
        # Constraints
        g1 = -x[0] + 0.0193 * x[2]
        g2 = -x[1] + 0.00954 * x[2]
        g3 = -np.pi * x[2]**2 * x[3] - (4/3) * np.pi * x[2]**3 + 1296000
        g4 = x[3] - 240
        
        # Penalty for constraint violations
        penalty = 0
        for g in [g1, g2, g3, g4]:
            if g > 0:
                penalty += 1e6 * g  # Large penalty for violations
        
        return cost + penalty

# Levy flight function
def Levy(dim):
    beta = 1.5
    sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) / 
             (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = 0.01 * np.random.randn(dim) * sigma
    v = np.random.randn(dim)
    zz = np.power(np.absolute(v), (1 / beta))
    step = np.divide(u, zz)
    return step

# SRA Algorithm
def SRA24(objf, lb, ub, dim, PopSize, iters, function_name):
    s = solution()
    
    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim
    lb = np.array(lb)
    ub = np.array(ub)
    
    L = 0.5
    h = 6.625e-34
    h2 = h**2
    Cost = np.full(PopSize, float("inf"))
    pos = np.zeros((PopSize, dim))
    Psai = np.zeros((PopSize, dim))
    for i in range(dim):
        pos[:, i] = np.random.uniform(0, 1, PopSize) * (ub[i] - lb[i]) + lb[i]
    
    for i in range(PopSize):
        for j in range(dim):
            pos[i, j] = np.clip(pos[i, j], lb[j], ub[j])
            Psai[i, j] = np.sin(pos[i, j])
        Cost[i] = objf.evaluate(pos[i, :])
    
    SmellOrder = np.sort(Cost)
    SmellIndex = np.argsort(Cost)
    Worst_Cost = SmellOrder[PopSize - 1]
    Best_Cost = SmellOrder[0]
    sorted_population = pos[SmellIndex, :]
    Best_X = sorted_population[0, :]
    sorted_Psai = Psai[SmellIndex, :]
    Best_Psai = sorted_Psai[0, :]
    Worst_Psai = sorted_Psai[PopSize - 1, :]
    
    convergence_curve = np.zeros(iters)
    
    timerStart = time.time()
    s.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    
    for l in range(iters):
        b = 1 - (l ** (1.0 / 5)) / (iters ** (1.0 / 5))
        
        SmellOrder = np.sort(Cost)
        SmellIndex = np.argsort(Cost)
        sorted_Psai = Psai[SmellIndex, :]
        Best_Psai = sorted_Psai[0, :]
        Worst_Psai = sorted_Psai[PopSize - 1, :]
        
        Seq = np.array(range(PopSize))
        R = PopSize - Seq
        p = np.sqrt(R / PopSize)
        
        for i in range(PopSize):
            h2 = p[i]
            Xnew = np.zeros(dim)
            vc = np.random.uniform(-b, b, dim)
            Z = Levy(dim)
            k = 1
            if random.random() < 0.03:
                Xnew = (ub - lb) * np.random.uniform(0, 1, size=dim) + lb
            else:
                ids_except_current = [_ for _ in range(PopSize) if _ != i]
                id_1, id_2 = random.sample(ids_except_current, 2)
                Threshold = (l / iters) ** 3
                if np.abs(p[i]) < Threshold:
                    if np.random.rand() < 0.5:
                        Xnew = k * random.random() + 2 * pos[i, :] - pos[i - 1, :]
                    else:
                        Xnew = Best_X - 0.1 * Z + np.random.rand() * ((ub - lb) * np.random.rand() + lb)
                else:
                    pos_1 = Best_X + random.random() * vc * (h * (Best_Psai - Worst_Psai) + 
                            (h2 * (Psai[id_1, :] - 2 * Psai[i, :] + Psai[id_2, :]))) / Psai[i, :]
                    pos_2 = pos[i, :] + random.random() * vc * (h * (Best_Psai - Worst_Psai) + 
                            (h2 * (Psai[id_1, :] + 2 * Psai[i, :] + Psai[id_2, :]))) / Psai[i, :]
                    Xnew = np.where( np.random.uniform(0, 1) < p[i], pos_1, pos_2)
            
            Xnew = np.clip(Xnew, lb, ub)
            
            Xnew_Cost = objf.evaluate(Xnew)
            if Cost[i] > Xnew_Cost:
                Cost[i] = Xnew_Cost
                pos[i, :] = Xnew
                if Cost[i] < Best_Cost:
                    Best_X = pos[i, :]
                    Best_Cost = Cost[i]
            if Cost[i] > Worst_Cost:
                Worst_Cost = Cost[i]
            
            Psai[i, :] = np.sin(random.random() * pos[i, :])
        
        convergence_curve[l] = Best_Cost
    
    timerEnd = time.time()
    s.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime = timerEnd - timerStart
    s.convergence = convergence_curve
    s.optimizer = "SRA"
    s.objfname = function_name
    s.best = Best_Cost
    s.bestIndividual = Best_X
    
    return s

# Main execution
def main():
    # Define bounds for variables [Ts, Th, R, L]
    bounds = [(0.0625, 99 * 0.0625), (0.0625, 99 * 0.0625), (10, 200), (10, 200)]
    lb = [b[0] for b in bounds]
    ub = [b[1] for b in bounds]
    
    # Parameters
    PopSize = 20
    iters = 2500
    n_runs = 30
    dim = 4
    function_name = "Pressure Vessel Design"
    
    # Objective function
    objf = ObjectiveFunction()
    
    # Store results
    best_fitnesses = []
    best_solutions = []
    
    # Run SRA multiple times
    for run in range(n_runs):
        s = SRA24(objf, lb, ub, dim, PopSize, iters, function_name)
        best_fitnesses.append(s.best)
        best_solutions.append(s.bestIndividual)
        
        # Check constraints for the best solution
        x = s.bestIndividual
        g1 = -x[0] + 0.0193 * x[2]
        g2 = -x[1] + 0.00954 * x[2]
        g3 = -np.pi * x[2]**2 * x[3] - (4/3) * np.pi * x[2]**3 + 1296000
        g4 = x[3] - 240
        constr = [g1, g2, g3, g4]
        if all(c <= 0 for c in constr):
            print(f"Run {run + 1}: Valid solution, Fitness = {s.best:.4f}")
        else:
            print(f"Run {run + 1}: Invalid solution, Fitness = {s.best:.4f}")
    
    # Calculate statistics
    best_fitnesses = np.array(best_fitnesses)
    best_idx = np.argmin(best_fitnesses)
    worst_idx = np.argmax(best_fitnesses)
    
    print("\n=== Results ===")
    print(f"Best Fitness: {best_fitnesses[best_idx]:.4f}")
    print(f"Worst Fitness: {best_fitnesses[worst_idx]:.4f}")
    print(f"Mean Fitness: {np.mean(best_fitnesses):.4f}")
    print(f"Standard Deviation: {np.std(best_fitnesses):.4f}")
    
    # Print best parameters
    best_params = best_solutions[best_idx]
    print("\nBest Parameters:")
    print(f"Ts (x1): {best_params[0]:.4f} inches")
    print(f"Th (x2): {best_params[1]:.4f} inches")
    print(f"R (x3): {best_params[2]:.4f} inches")
    print(f"L (x4): {best_params[3]:.4f} inches")
    
    # Verify constraints for best solution
    constr = [g1, g2, g3, g4]
    print("\nConstraint Values (should be <= 0):")
    for i, c in enumerate(constr, 1):
        print(f"g{i}: {c:.4f}")

if __name__ == "__main__":
    np.random.seed(42)  # For reproducibility
    main()

Run 1: Valid solution, Fitness = 7155.5267
Run 2: Valid solution, Fitness = 6029.6069
Run 3: Valid solution, Fitness = 6335.9099
Run 4: Valid solution, Fitness = 6093.9052
Run 5: Valid solution, Fitness = 5954.2965
Run 6: Valid solution, Fitness = 6143.9826
Run 7: Valid solution, Fitness = 6523.6213
Run 8: Valid solution, Fitness = 5937.9032
Run 9: Valid solution, Fitness = 6309.0825
Run 10: Valid solution, Fitness = 7006.3483
Run 11: Valid solution, Fitness = 6147.4331
Run 12: Valid solution, Fitness = 6856.2525
Run 13: Valid solution, Fitness = 6166.2844
Run 14: Valid solution, Fitness = 6234.6920
Run 15: Valid solution, Fitness = 6407.8192
Run 16: Valid solution, Fitness = 6011.1176
Run 17: Valid solution, Fitness = 5903.1379
Run 18: Valid solution, Fitness = 6193.3229
Run 19: Valid solution, Fitness = 6352.1889
Run 20: Valid solution, Fitness = 5995.0169
Run 21: Valid solution, Fitness = 5889.3863
Run 22: Valid solution, Fitness = 7310.3046
Run 23: Valid solution, Fitness = 6427.29